### Imports

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from dataset import PACSDataset
from vit_grqo import ViTGRQO, grqo_loss_from_gradients

### CONFIG 

In [2]:
import os
from datasets import load_dataset
from PIL import Image

# Your constants
DATA_ROOT = "../../../pacs_data/pacs_data"
DOMAINS = ["art_painting", "cartoon", "photo", "sketch"]
CLASSES = ["dog", "elephant", "giraffe", "guitar", "horse", "house", "person"]
BATCH_SIZE = 24
NUM_CLASSES = 7
NUM_EPOCHS = 5
LR = 1e-4
TOPK = 16
ALPHA = 1.0
BETA = 0.5
TAU = 1e-3

# dataset = load_dataset("flwrlabs/pacs", split="train")

# os.makedirs(DATA_ROOT, exist_ok=True)
# for domain in DOMAINS:
#     for cls in CLASSES:
#         os.makedirs(f"{DATA_ROOT}/{domain}/{cls}", exist_ok=True)

# for i, example in enumerate(dataset):
#     domain = example["domain"]  
#     label_idx = example["label"]  
#     label = CLASSES[label_idx]

#     if domain not in DOMAINS:
#         raise ValueError(f"Unexpected domain: {domain}. Expected one of {DOMAINS}")
#     if label not in CLASSES:
#         raise ValueError(f"Unexpected label: {label}. Expected one of {CLASSES}")
    
#     image = example["image"]
#     image.save(f"{DATA_ROOT}/{domain}/{label}/image_{i}.jpg")


### Data

In [3]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

pacs_data = PACSDataset(DATA_ROOT, DOMAINS, transform, BATCH_SIZE)
ALL_DOMAINS = ['photo', 'art_painting', 'cartoon', 'sketch']
LEAVE_OUT = 'sketch'  
TRAIN_DOMAINS = [d for d in ALL_DOMAINS if d != LEAVE_OUT]
VAL_DOMAIN = LEAVE_OUT

train_datasets = [pacs_data.get_dataloader(domain=d, train=True) for d in TRAIN_DOMAINS]
train_loader = torch.utils.data.DataLoader(
    torch.utils.data.ConcatDataset([d.dataset for d in train_datasets]),
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Validation loader
val_loader = pacs_data.get_dataloader(domain=VAL_DOMAIN, train=False)

### Model 

In [4]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ViTGRQO(num_classes=NUM_CLASSES, topk=TOPK)
model.to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=0.05)
criterion = nn.CrossEntropyLoss()

### Finetuning

In [5]:
for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    running_samples = 0
    
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        # Forward pass
        logits, patch_tokens, token_probs = model(images)
        
        # Classification + GRQO loss
        cls_loss = criterion(logits, labels)
        grqo_loss = grqo_loss_from_gradients(
            logits, patch_tokens, token_probs, labels,
            teacher_probs=model.teacher_probs,
            alpha=ALPHA, beta=BETA, topk=TOPK, tau=TAU
        )
        loss = cls_loss + grqo_loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # EMA teacher update
        with torch.no_grad():
            if model.teacher_probs.numel() != token_probs.mean(dim=0).numel():
                model.teacher_probs = token_probs.mean(dim=0, keepdim=True)
            else:
                model.teacher_probs = 0.9*model.teacher_probs + 0.1*token_probs.mean(dim=0, keepdim=True)
        
        # Track running loss and accuracy
        running_loss += loss.item() * images.size(0)
        preds = logits.argmax(dim=1)
        running_corrects += (preds == labels).sum().item()
        running_samples += labels.size(0)
    
    # Compute epoch metrics
    epoch_loss = running_loss / running_samples
    epoch_acc = running_corrects / running_samples
    
    # ------------------- VALIDATION -----------------
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            logits, _, _ = model(images)
            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    val_acc = correct / total
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] - "
          f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, "
          f"Val Acc (LODO on {VAL_DOMAIN}): {val_acc:.4f}")


Epoch [1/5] - Train Loss: 0.6694, Train Acc: 0.8276, Val Acc (LODO on sketch): 0.2799
Epoch [2/5] - Train Loss: 0.1007, Train Acc: 0.9732, Val Acc (LODO on sketch): 0.2901
Epoch [3/5] - Train Loss: 0.0376, Train Acc: 0.9913, Val Acc (LODO on sketch): 0.4555
Epoch [4/5] - Train Loss: 0.0349, Train Acc: 0.9911, Val Acc (LODO on sketch): 0.4020
Epoch [5/5] - Train Loss: 0.0633, Train Acc: 0.9837, Val Acc (LODO on sketch): 0.3397


In [ ]:
# ------------------- LODO Experiments -------------------
ALL_DOMAINS = ['photo', 'art_painting', 'cartoon', 'sketch']
lodo_results = {}

for LEAVE_OUT in ALL_DOMAINS:
    print(f"\n=== LODO: Leaving out domain '{LEAVE_OUT}' ===")
    TRAIN_DOMAINS = [d for d in ALL_DOMAINS if d != LEAVE_OUT]
    VAL_DOMAIN = LEAVE_OUT

    # Train loader
    train_datasets = [pacs_data.get_dataloader(domain=d, train=True) for d in TRAIN_DOMAINS]
    train_loader = torch.utils.data.DataLoader(
        torch.utils.data.ConcatDataset([d.dataset for d in train_datasets]),
        batch_size=BATCH_SIZE,
        shuffle=True
    )

    # Validation loader
    val_loader = pacs_data.get_dataloader(domain=VAL_DOMAIN, train=False)

    # Reset model & optimizer for this LODO split
    model = ViTGRQO(num_classes=NUM_CLASSES, vit_model='vit_small_patch16_224', topk=TOPK).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(NUM_EPOCHS):
        # TRAIN
        model.train()
        running_loss = 0.0
        running_corrects = 0
        running_samples = 0

        for images, labels in train_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            logits, patch_tokens, token_probs = model(images)
            cls_loss = criterion(logits, labels)
            grqo_loss = grqo_loss_from_gradients(
                logits, patch_tokens, token_probs, labels,
                teacher_probs=model.teacher_probs,
                alpha=ALPHA, beta=BETA, topk=TOPK, tau=TAU
            )
            loss = cls_loss + grqo_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # EMA teacher update
            with torch.no_grad():
                teacher_update = token_probs.mean(dim=0, keepdim=True)
                if model.teacher_probs.numel() != teacher_update.numel():
                    model.teacher_probs = teacher_update
                else:
                    model.teacher_probs = 0.9 * model.teacher_probs + 0.1 * teacher_update

            running_loss += loss.item() * images.size(0)
            preds = logits.argmax(dim=1)
            running_corrects += (preds == labels).sum().item()
            running_samples += labels.size(0)

        epoch_loss = running_loss / running_samples
        epoch_acc = running_corrects / running_samples

        # VALIDATION
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                logits, _, _ = model(images)
                preds = logits.argmax(dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_acc = correct / total
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] - "
              f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, "
              f"Val Acc (LODO on {VAL_DOMAIN}): {val_acc:.4f}")

    # Store final validation accuracy
    lodo_results[VAL_DOMAIN] = val_acc

# ------------------- ALL-DOMAINS Baseline -------------------
print("\n=== Baseline: Train & Validate on All Domains ===")

# Concatenate all train datasets
all_train_datasets = [pacs_data.get_dataloader(domain=d, train=True) for d in ALL_DOMAINS]
train_loader_all = torch.utils.data.DataLoader(
    torch.utils.data.ConcatDataset([d.dataset for d in all_train_datasets]),
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Concatenate all validation datasets
all_val_datasets = [pacs_data.get_dataloader(domain=d, train=False) for d in ALL_DOMAINS]
val_loader_all = torch.utils.data.DataLoader(
    torch.utils.data.ConcatDataset([d.dataset for d in all_val_datasets]),
    batch_size=BATCH_SIZE,
    shuffle=False
)

# Reset model & optimizer
model = ViTGRQO(num_classes=NUM_CLASSES, vit_model='vit_small_patch16_224.augreg_in21k', topk=TOPK).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    running_samples = 0

    for images, labels in train_loader_all:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        logits, patch_tokens, token_probs = model(images)
        cls_loss = criterion(logits, labels)
        grqo_loss = grqo_loss_from_gradients(
            logits, patch_tokens, token_probs, labels,
            teacher_probs=model.teacher_probs,
            alpha=ALPHA, beta=BETA, topk=TOPK, tau=TAU
        )
        loss = cls_loss + grqo_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            teacher_update = token_probs.mean(dim=0, keepdim=True)
            if model.teacher_probs.numel() != teacher_update.numel():
                model.teacher_probs = teacher_update
            else:
                model.teacher_probs = 0.9 * model.teacher_probs + 0.1 * teacher_update

        running_loss += loss.item() * images.size(0)
        preds = logits.argmax(dim=1)
        running_corrects += (preds == labels).sum().item()
        running_samples += labels.size(0)

    epoch_loss = running_loss / running_samples
    epoch_acc = running_corrects / running_samples

    # VALIDATION
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader_all:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            logits, _, _ = model(images)
            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] - "
          f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, "
          f"Val Acc (All Domains): {val_acc:.4f}")

# ------------------- Summary -------------------
print("\n=== Summary of Results ===")
print("LODO Results:")
for domain, acc in lodo_results.items():
    print(f"{domain}: {acc:.4f}")
print(f"All-domains baseline: {val_acc:.4f}")



=== LODO: Leaving out domain 'photo' ===
Epoch [1/5] - Train Loss: 0.9525, Train Acc: 0.6710, Val Acc (LODO on photo): 0.9461
Epoch [2/5] - Train Loss: 0.2527, Train Acc: 0.9150, Val Acc (LODO on photo): 0.9701
Epoch [3/5] - Train Loss: 0.1053, Train Acc: 0.9680, Val Acc (LODO on photo): 0.9611
Epoch [4/5] - Train Loss: 0.1106, Train Acc: 0.9639, Val Acc (LODO on photo): 0.9611
Epoch [5/5] - Train Loss: 0.0646, Train Acc: 0.9781, Val Acc (LODO on photo): 0.9581

=== LODO: Leaving out domain 'art_painting' ===
Epoch [1/5] - Train Loss: 0.7919, Train Acc: 0.7472, Val Acc (LODO on art_painting): 0.7976
Epoch [2/5] - Train Loss: 0.2231, Train Acc: 0.9235, Val Acc (LODO on art_painting): 0.8439
Epoch [3/5] - Train Loss: 0.1053, Train Acc: 0.9663, Val Acc (LODO on art_painting): 0.7561
Epoch [4/5] - Train Loss: 0.0743, Train Acc: 0.9766, Val Acc (LODO on art_painting): 0.7512
Epoch [5/5] - Train Loss: 0.0454, Train Acc: 0.9854, Val Acc (LODO on art_painting): 0.8537

=== LODO: Leaving out d